In [2]:
import mysql.connector
import pandas as pd
from nltk.corpus import stopwords
import gensim
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from fuzzywuzzy import process
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

C:\Users\Andre\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
C:\Users\Andre\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
scraping()

Iniciando scraping do site G1. Isto pode demorar um pouco...
Buscando notícias e salvando em lista...
Iniciando scraping do site Google News...
Buscando notícias e salvando em lista...
Iniciando scraping do site Estadão...
Buscando notícias e salvando em lista...
Importando listas para o banco de dados...


In [3]:
db = mysql.connector.connect(
   host="database-1.cpqjjsrzpykc.us-east-2.rds.amazonaws.com",
   user="admin",
   passwd="Andre.2021",
   database="noticias"
)

print(db)

In [4]:
cursor = db.cursor()
query = ("SELECT * FROM noticias.news")
cursor.execute(query)
records = cursor.fetchall()
db.close()

In [5]:
records[0]

(1,
 'De olho em 2022, presidente diverge da equipe econômica sobre valor',
 'Governo vai jogar a discussão para deputados forçarem benefício maior.',
 'Há 6 horas',
 'https://g1.globo.com/politica/blog/julia-duailibi/post/2021/08/09/mp-do-auxilio-brasil-vai-chegar-ao-congresso-sem-valor-do-beneficio-equipe-economica-briga-por-r-300.ghtml',
 'https://s2.glbimg.com/DCgY1ouev_mcFNaDY5EZxq0BkqE=/0x226:3618x2261/540x304/smart/filters:max_age(3600)/https://i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2021/r/x/SV1ZvKRUqgxRKiCGX0pQ/2021-08-09t154442z-752664168-rc2n1p9n58zx-rtrmadp-3-brazil-politics.jpg',
 'G1')

In [31]:
news_df = pd.DataFrame(records, columns=['id','titulo','subtitulo','hora','link','img', 'fonte'])
news_df = news_df.drop(columns=['id'])

In [32]:
news_df['titulo'].isnull().sum()

0

In [33]:
news_df.shape

(114, 6)

In [34]:
def process_texto(s):
    
    # seleciona apenas letras (lembrando que o texto está em português e as letras possuem acento)
    texto_limpo = gensim.utils.simple_preprocess(s)
    
    # remove stopwords
    sw = stopwords.words('portuguese')
    sem_stopwords = [word for word in texto_limpo if word not in sw]
    
    return sem_stopwords

In [35]:
news_df['titulo_sw'] = news_df['titulo'].apply(lambda s: process_texto(s)).apply(lambda x: ' '.join(x))

In [36]:
news_df.head(2)

,titulo,subtitulo,hora,link,img,fonte,titulo_sw
0,"Alison e Álvaro perdem, e Brasil fica sem meda...",,Há 2 horas,https://ge.globo.com/olimpiadas/noticia/alison...,https://s2.glbimg.com/x85VfdtXMrfxpMDqclxBVoRh...,G1,alison álvaro perdem brasil fica medalha vôlei...
1,ASSISTA: Dora Varella e Yndiara Asp avançam no...,,Há 1 hora,https://ge.globo.com/olimpiadas/ao-vivo/olimpi...,https://s2.glbimg.com/XqpZlXZP1s_AlVY6QKKEAwDN...,G1,assista dora varella yndiara asp avançam skate...


In [37]:
# Bag-of-Words (bow) vectorises the data
# transforming tokens into useful features
     
bow_transformer = CountVectorizer().fit_transform(news_df['titulo_sw'])

news_tfidf = TfidfTransformer().fit_transform(bow_transformer)


In [38]:
nn = NearestNeighbors(metric='cosine', n_neighbors=4)

In [39]:
# Recomendador de notícias

lista = []
def recommender(noticia):
    lista.clear()
    nn.fit(news_tfidf)
    idx=process.extractOne(noticia, news_df['titulo'])[2]
    print('Notícias recomendadas: ')
    distances, indices=nn.kneighbors(news_tfidf[idx], n_neighbors=4)
    n=0
    for i in indices:
        lista.append(i)
        
    for n in lista[0][1:]:
        print(f'{news_df.iloc[n]} \n\n')
        

In [40]:
recommender(news_df['titulo'][1])

Notícias recomendadas: 
titulo       Olimpíada de Tóquio 2021: Brasil retoma disput...
subtitulo                                                     
hora                                             6 horas atrás
link         news.google.com/articles/CBMiLWh0dHBzOi8vd3d3L...
img          https://lh3.googleusercontent.com/proxy/dYbBAD...
fonte                                               BBC Brasil
titulo_sw    olimpíada tóquio brasil retoma disputas skate ...
Name: 99, dtype: object 


titulo       Medalhista de novo: assista ao salto que deu o...
subtitulo                                                     
hora                                               Há 16 horas
link         https://g1.globo.com/mundo/video/thiago-braz-v...
img                                                           
fonte                                                       G1
titulo_sw    medalhista novo assista salto deu bronze thiag...
Name: 17, dtype: object 


titulo       Cachorro 'fã do Jornal Naci